## 1. Setup and Installation

In [ ]:
# Install required libraries
!pip install -q discord.py groq python-dotenv PyMuPDF

## 2. Imports and API Key Configuration

In [ ]:
import discord
from discord import app_commands
import os
import asyncio
import fitz  # PyMuPDF
from groq import Groq, AsyncGroq
from google.colab import userdata
from typing import Optional

# --- Load Secrets ---
# Load the discord bot token and API (groq) key in the secrets key section of Google Colab

try:
    DISCORD_BOT_TOKEN = userdata.get('DISCORD_BOT_TOKEN')
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("✅ Secrets loaded successfully.")
except Exception as e:
    print(f"🚨 Error loading secrets: {e}. Please ensure DISCORD_BOT_TOKEN and GROQ_API_KEY are set in Colab Secrets.")

# --- Configuration ---
MAX_FILE_SIZE = 15 * 1024 * 1024  # 15 MB

## 3. Groq AI Client

In [ ]:
class GroqClient:
    def __init__(self, api_key):
        self.client = AsyncGroq(api_key=api_key)
        self.model = "llama3-70b-8192" # Model

    async def generate(self, messages: list, max_tokens=4096) -> str:
        """Generates a response from the Groq API using a message history."""
        try:
            chat_completion = await self.client.chat.completions.create(
                messages=messages,
                model=self.model,
                temperature=0.7,
                max_tokens=max_tokens,
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            print(f"🚨 Groq API Error: {e}")
            return "Sorry, there was an error connecting to the AI service."

# Instantiate the client
groq_client = GroqClient(api_key=GROQ_API_KEY)
print("🤖 Groq AI Client Initialized.")

## 4. Discord Bot and Helper Functions

In [ ]:
# --- Discord Bot and Helper Functions ---

# Bot setup with necessary intents
intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents=intents)
tree = app_commands.CommandTree(client)

# Attach the conversation history to the client instance for answering followup questions
client.conversation_history = {}

async def get_text_from_attachment(attachment: discord.Attachment) -> Optional[str]:
    """Extracts text from a discord.Attachment (supports PDF, TXT)."""
    if attachment.size > MAX_FILE_SIZE:
        return "❌ File is too large. The maximum size is 15MB."

    file_bytes = await attachment.read()
    try:
        if attachment.content_type == "application/pdf":
            with fitz.open(stream=file_bytes, filetype="pdf") as doc:
                return "".join(page.get_text() for page in doc)
        else:
            return file_bytes.decode('utf-8')
    except Exception as e:
        print(f"Error reading file: {e}")
        return f"❌ Error reading file: {e}"

async def send_long_message(interaction: discord.Interaction, content: str, title: str):
    """Sends long messages in chunks using embeds."""
    embed = discord.Embed(title=f"**{title}**", color=discord.Color.blue())
    for i in range(0, len(content), 4096):
        chunk = content[i:i+4096]
        embed.description = chunk
        if i == 0:
            await interaction.followup.send(embed=embed)
        else:
            await interaction.channel.send(embed=embed)

async def send_long_reply(message: discord.Message, content: str):
    """Sends a long reply by splitting it into chunks."""
    if len(content) <= 2000:
        await message.reply(content)
        return

    chunks = [content[i:i+1990] for i in range(0, len(content), 1990)]
    first = True
    for chunk in chunks:
        if first:
            await message.reply(chunk)
            first = False
        else:
            await message.channel.send(chunk)

## 5. Bot Events (Ready, Greetings, General Q&A)

In [ ]:
# --- Bot Events (Ready, Greetings, General Q&A) ---

@client.event
async def on_ready():
    await tree.sync()
    print(f'✅ Logged in as {client.user}')
    print('🚀 ClassBuilder is online and ready for Slash Commands!')

@client.event
async def on_message(message):
    # Ignore messages from the bot itself
    if message.author == client.user:
        return

    # Handle greetings
    greetings = ["hi", "hello", "hey"]
    if message.content.lower() in greetings:
        # Clear memory on a new greeting
        if message.channel.id in client.conversation_history:
            del client.conversation_history[message.channel.id]

        embed = discord.Embed(
            title="👋 Hello! I'm the ClassBuilder AI Chatbot.",
            description="I can help you with your study materials. Here are my commands:",
            color=discord.Color.dark_teal()
        )
        embed.add_field(name="`/summary`", value="Generates a summary of your text or PDF.", inline=False)
        embed.add_field(name="`/notes`", value="Creates structured, pointwise notes.", inline=False)
        embed.add_field(name="`/quiz`", value="Generates a practice quiz with answers.", inline=False)
        embed.add_field(name="`/export`", value="Bundles a summary, notes, and quiz into a file.", inline=False)
        embed.add_field(name="`/clear`", value="Deletes a specified number of recent messages.", inline=False)
        await message.channel.send(embed=embed)
        return

    # Handle general questions when the bot is mentioned
    if client.user.mentioned_in(message):
        async with message.channel.typing():
            # Get or create the history for this channel
            history = client.conversation_history.setdefault(message.channel.id, [])

            # Add the new user message to history
            user_prompt = message.content.replace(f'<@{client.user.id}>', '').strip()
            history.append({"role": "user", "content": user_prompt})

            # Keep the history to the last 10 messages
            history = history[-10:]
            client.conversation_history[message.channel.id] = history

            # Prepare the full payload for the API
            system_prompt = {"role": "system", "content": "You are a helpful general knowledge assistant. Answer the user's question clearly and concisely based on the conversation history."}
            messages_with_system_prompt = [system_prompt] + history

            # Get the response from Groq
            response = await groq_client.generate(messages_with_system_prompt)

            # Add the bot's response to history
            history.append({"role": "assistant", "content": response})

            await send_long_reply(message, response)

## 6. Slash Command Definitions

In [ ]:
# Summary
@tree.command(name="summary", description="Generates a summary of your text or PDF.")
@app_commands.describe(word_count="How long should the summary be (e.g., 100)?", text="Text to summarize.", attachment="File to summarize.")
async def summary(interaction: discord.Interaction, word_count: int = 100, text: Optional[str] = None, attachment: Optional[discord.Attachment] = None):
    await interaction.response.defer()

    if not text and not attachment:
        await interaction.followup.send("Please provide either text or attach a file.")
        return

    content = text or await get_text_from_attachment(attachment)
    if content.startswith("❌"):
        await interaction.followup.send(content)
        return

    system_prompt = "You are a highly skilled summarization engine."
    user_prompt = f"Summarize the following text in approximately {word_count} words:\n\n---\n{content}"

    # Correctly format the messages into a list
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    result = await groq_client.generate(messages)
    await send_long_message(interaction, result, "📝 Summary")

# Notes
@tree.command(name="notes", description="Creates structured, pointwise notes from your text or PDF.")
@app_commands.describe(text="Text to turn into notes.", attachment="File to turn into notes.")
async def notes(interaction: discord.Interaction, text: Optional[str] = None, attachment: Optional[discord.Attachment] = None):
    await interaction.response.defer()

    if not text and not attachment:
        await interaction.followup.send("Please provide either text or attach a file.")
        return

    content = text or await get_text_from_attachment(attachment)
    if content.startswith("❌"):
        await interaction.followup.send(content)
        return

    system_prompt = "You are an expert at creating structured study notes. You must use bullet points."
    user_prompt = f"Extract the key information from the following text and present it as clear, concise, pointwise notes. If possible, generate 10 or more points if the source material is sufficient:\n\n---\n{content}"

    # Correctly format the messages into a list
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    result = await groq_client.generate(messages)
    await send_long_message(interaction, result, "📚 Notes")

# Quiz
@tree.command(name="quiz", description="Generates a practice quiz from your text or PDF.")
@app_commands.describe(num_questions="How many questions would you like?", text="Text to create a quiz from.", attachment="File to create a quiz from.")
async def quiz(interaction: discord.Interaction, num_questions: int = 5, text: Optional[str] = None, attachment: Optional[discord.Attachment] = None):
    await interaction.response.defer()

    if not text and not attachment:
        await interaction.followup.send("Please provide either text or attach a file.")
        return

    content = text or await get_text_from_attachment(attachment)
    if content.startswith("❌"):
        await interaction.followup.send(content)
        return

    system_prompt = "You are a quiz generation expert for academic and educational material. Your task is to create questions based ONLY on the main educational topics, concepts, and definitions in the provided text. You must ignore any metadata, such as author names, professor names, copyright notices, or course logistics. Focus strictly on the subject matter. Create questions with clear answers from the text and hide the answers with Discord's spoiler tags (||answer||)."
    user_prompt = f"Generate a quiz with exactly {num_questions} questions from the following text. Remember to ignore metadata and focus only on the educational content. If you cannot generate the requested number of questions due to insufficient information, generate as many as you can and add a note at the very end explaining this (e.g., 'Note: I could only generate X questions from the provided text.'). Format as Q1, A1, Q2, A2, etc. and put all answers in spoiler tags.\n\n---\n{content}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    result = await groq_client.generate(messages)
    await send_long_message(interaction, result, "❓ Quiz")

# Export
@tree.command(name="export", description="Exports summary, notes, and quiz into a file.")
@app_commands.describe(format="The file format to export to.", text="Text to process.", attachment="File to process.")
@app_commands.choices(format=[
    app_commands.Choice(name="Text File", value="txt"),
    app_commands.Choice(name="JSON File", value="json"),
])
async def export(interaction: discord.Interaction, format: app_commands.Choice[str], text: Optional[str] = None, attachment: Optional[discord.Attachment] = None):
    await interaction.response.defer()

    if not text and not attachment:
        await interaction.followup.send("Please provide either text or attach a file.")
        return

    content = text or await get_text_from_attachment(attachment)
    if content.startswith("❌"):
        await interaction.followup.send(content)
        return

    # Generate content with correctly formatted API calls
    summary = await groq_client.generate([{"role": "user", "content": f"Summarize this text concisely: {content}"}])
    notes = await groq_client.generate([{"role": "user", "content": f"Create bullet-point notes from this text: {content}"}])
    quiz = await groq_client.generate([{"role": "user", "content": f"Create a 5-question quiz from this text with answers in spoiler tags: {content}"}])

    # Prepare data for export
    export_data = {"summary": summary, "notes": notes, "quiz": quiz}

    if format.value == "txt":
        file_content = f"--- SUMMARY ---\n{summary}\n\n--- NOTES ---\n{notes}\n\n--- QUIZ ---\n{quiz}"
        filename = "export.txt"
    elif format.value == "json":
        import json
        file_content = json.dumps(export_data, indent=4)
        filename = "export.json"

    # Send the file
    import io
    file = discord.File(io.BytesIO(file_content.encode('utf-8')), filename=filename)
    await interaction.followup.send("Here is your exported file:", file=file)

# Clear
@tree.command(name="clear", description="Deletes a specified number of recent messages.")
@app_commands.describe(amount="The number of messages to delete (e.g., 100).")
@app_commands.checks.has_permissions(manage_messages=True)
async def clear(interaction: discord.Interaction, amount: int):
    if amount <= 0:
        await interaction.response.send_message("Please provide a number greater than 0.", ephemeral=True)
        return

    await interaction.response.defer(ephemeral=True)
    deleted = await interaction.channel.purge(limit=amount)
    await interaction.followup.send(f"✅ Successfully deleted {len(deleted)} messages.", ephemeral=True)

@clear.error
async def clear_error(interaction: discord.Interaction, error: app_commands.AppCommandError):
    if isinstance(error, app_commands.MissingPermissions):
        await interaction.response.send_message("❌ You don't have the `Manage Messages` permission to use this command.", ephemeral=True)


## 7. Main Execution Block

In [ ]:
# --- Main Execution ---
if not DISCORD_BOT_TOKEN or not GROQ_API_KEY:
    print("🚨 FATAL ERROR: Secrets not found. Please follow the instructions in Cell 2.")
else:
    try:
        await client.start(DISCORD_BOT_TOKEN)
    except discord.errors.LoginFailure:
        print("🚨 FATAL ERROR: Invalid Discord token.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")